In [1]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =20

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

# MODEL CONFIG
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
os.chdir("data")

In [3]:
df = pd.read_csv("CleanedData.csv")

In [4]:
df["Category_name"].value_counts()

tv, television, led                2266
drive, storage, hard               2105
headphone, earbud, bluetooth       1801
tv, 4k, television                 1636
headphone                          1580
receiver, amplifier, home          1541
car, gps, dash                     1464
camera, shoot, point               1386
tablet, computer, ipad             1363
camera, mirrorless                 1302
laptop, computer                   1210
speaker, portable, bluetooth       1202
player, ipod                       1098
sound, speaker, home               1088
car, speaker, subwoofer            1072
speaker, audio                     1035
case, computer                      926
laptop, computer, macbook           925
speaker, bluetooth, mobile          841
phone, cell                         789
mount, tv, stand                    783
camera, camcorder, action           750
wireless, speaker, audio            696
lens, camera                        683
monitor, computer                   654


## Price Elasticity for all Products.

In [5]:
#dfx = df[df["Category_name"].str.contains("speaker",case=False)]
dfx= df[(df["Year"]==2017) | (df["Year"]==2018)]

In [6]:
dfx = dfx.reset_index(drop=True)

In [7]:
dfx.sort_values(by=['Date'],ignore_index=True)

,id,Name,Brand,Description,Category_name,Merchant,Actual_price,Discount_price,isSale,Discount_per,Imp_count,Day,Month,Year,Week,Month_Name,Day_Name,Year_Month,Year_Week,Date
0,AVpfO3piLJeJML43...,KS9800-Series 65...,Samsung,"TV, Video & Home...","tv, 4k, television",Walmart.com,3497.99,3497.99,No,0.0,1,5,1,2017,1,January,Thursday,2017-01,"2017,01",2017-01-05
1,AVpfiNW81cnluZ0-...,Yamaha R-N301 Ne...,Yamaha,"Electronics,Home...","receiver, amplif...",Walmart.com,473.65,473.65,No,0.0,1,5,1,2017,1,January,Thursday,2017-01,"2017,01",2017-01-05
2,AVpgQiFNLJeJML43...,XDP-100R Portabl...,Pioneer,TVs Entertainmen...,"player, ipod",Walmart.com,399.99,399.99,No,0.0,1,5,1,2017,1,January,Thursday,2017-01,"2017,01",2017-01-05
3,AVpgQiFNLJeJML43...,XDP-100R Portabl...,Pioneer,TVs Entertainmen...,"player, ipod",Walmart.com,599.00,599.00,No,0.0,1,7,1,2017,1,January,Saturday,2017-01,"2017,01",2017-01-07
4,AVpfWOIFLJeJML43...,Alpine - UTE - A...,Alpine,"Electronics,Home...","car, gps, dash",Walmart.com,137.99,137.99,No,0.0,1,7,1,2017,1,January,Saturday,2017-01,"2017,01",2017-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40646,AVpfWurLLJeJML43...,Source 530 Full ...,NZXT,"Computers,Cases ...","case, computer",bhphotovideo.com,99.90,99.90,No,0.0,1,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25
40647,AV1YFizWvKc47QAV...,1TB WD Black Mob...,WD,"Computers,Intern...","drive, storage, ...",bhphotovideo.com,69.99,69.99,No,0.0,7,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25
40648,AV1YFjGjGV-KLJ3a...,MyWirelessTV2 Mu...,Actiontec,Streaming Media ...,"tv, streaming, m...",bhphotovideo.com,199.99,199.99,No,0.0,1,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25
40649,AVpfJpzM1cnluZ0-...,SL-15 Floorstand...,Cerwin-Vega,"Floor Speakers,B...","speaker, audio",bhphotovideo.com,449.00,449.00,No,0.0,9,25,7,2018,30,July,Wednesday,2018-07,"2018,30",2018-07-25


In [8]:
#Grouping data by Name and Week number.
dfx_mod = dfx.groupby(['Name', "Year_Month"]).agg({'Discount_per':'mean','Imp_count':'sum',"Category_name":lambda x: pd.Series.mode(x)[0],"Date":lambda x: pd.Series.mode(x)[0]}).reset_index()


In [9]:
#Rearranging
dfx_mod = dfx_mod[["Name","Category_name","Discount_per","Imp_count","Year_Month","Date"]]

In [10]:
dfx_mod['Date']= pd.to_datetime(dfx_mod['Date'])

In [11]:

dfx_mod.sort_values("Year_Month",ignore_index=True)

,Name,Category_name,Discount_per,Imp_count,Year_Month,Date
0,Logitech - Logi ...,"case, computer",0.0,1,2017-01,2017-01-08
1,PMX840BT Bluetoo...,"microphone, musi...",0.0,2,2017-01,2017-01-26
2,Magellan Roadmat...,"car, gps, dash",0.0,4,2017-01,2017-01-20
3,Alpine - UTE - A...,"car, gps, dash",0.0,1,2017-01,2017-01-07
4,Dell - XPS 2-in-...,"laptop, computer",0.0,9,2017-01,2017-01-24
...,...,...,...,...,...,...
10839,PowerShot G7 X M...,"camera, shoot, p...",0.0,12,2018-07,2018-07-25
10840,DP-300F Fully Au...,"turntable, cd, a...",0.0,7,2018-07,2018-07-25
10841,Alpha a5100 Mirr...,"camera, mirrorless",0.0,20,2018-07,2018-07-25
10842,MG278Q 27 Widesc...,"monitor, computer",0.0,10,2018-07,2018-07-25


#### X-Values = PRICE

In [12]:

#New X dataframe.
x_new = dfx_mod.pivot(index="Year_Month", columns='Name' ,values='Discount_per')
print("Structure of DataFrame = ",x_new.shape)
x_new.shape

Structure of DataFrame =  (19, 1223)


(19, 1223)

In [13]:
x_new.head()

Name,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),4K Video Camera,...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power 

In [14]:
# FOR X VALUES
#Filling missing values with fillna + forward filling
x_new_ = pd.DataFrame(x_new.to_records())
x_new_.fillna(method='ffill', inplace=True)
x_new_.fillna(method='bfill', inplace=True)
x_new_.head()

,Year_Month,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power Bank - Si

#### Y-Values = DEMAND or IMPRESSIONS

In [15]:
#New Y dataframe.
y_new = dfx_mod.pivot(index='Year_Month', columns='Name', values='Imp_count')
print("Structure of DataFrame = ",y_new.shape)
y_new.shape


Structure of DataFrame =  (19, 1223)


(19, 1223)

In [16]:
y_new.sample(5)

Name,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),4K Video Camera,...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power 

In [17]:
# FOR Y VALUES
#Filling missing values with fillna + forward filling
y_new_ = pd.DataFrame(y_new.to_records())
y_new_.fillna(method='ffill', inplace=True)
y_new_.fillna(method='bfill', inplace=True)
y_new_.head(5)

,Year_Month,1080p Day/Night PTZ Speed Dome Camera with 5 to 61mm Auto Focus Lens,10TB G-DRIVE with Thunderbolt,"12 MacBook (Mid 2017, Gold)","12 MacBook (Mid 2017, Silver)",12.3 32GB Multi-Touch 2-in-1 Chromebook Plus,120 Bulb LED Video Light (Daylight),128GB iPod touch (Gold) (6th Generation),128GB iPod touch (Space Gray) (6th Generation),"13.3 MacBook Air (Mid 2017, Silver)",14 Checkpoint Friendly Laptop Case,"15-Volt 3 Amp 6.0 mm-3.0 mm AC Adapter for TOSHIBA Tecra, Satellite and Satellite Pro Series Laptops","15.4 MacBook Pro with Touch Bar (Late 2016, Silver)","15.4 MacBook Pro with Touch Bar (Late 2016, Space Gray)",1500-Watt Digital Infrared Heater with Wi-Fi Phone App - White,1506TSA Lock,151 SE Outdoor Environmental Speakers (White),17.3 WE72 7RJ Mobile Workstation,1TB G-Drive Mobile Hard Drive with Thunderbolt,1TB T5 Portable Solid-State Drive (Black),1TB WD Black Mobile OEM Hard Drive (WD10JPLX),"2) New Kicker 41KSC6934 6x9 3-Way 300 Watt Car Audio Coaxial Speakers KSC693""",21.5 iMac (Mid 2017),21.5 iMac with Retina 4K Display (Mid 2017),22 Widescreen Full HD 1080p LED Monitor,24-70mm f/2.8 GM Lens and 82mm Circular Polarizer Filter Kit,240GB SSD Plus SATA III 2.5 Internal SSD,251 Outdoor Environmental Speakers (White),27 iMac with Retina 5K Display (Mid 2017),2PJ4280 - Peerless-AV Wall Mount for Tablet PC,2TB My Passport USB 3.0 Secure Portable Hard Drive (Orange),2TB Red 5400 rpm SATA III 3.5 Internal NAS HDD,32GB High Speed UHS-I SDHC U3 Memory Card (Class 10),32GB iPod touch (Gold) (6th Generation),32GB iPod touch (Pink) (6th Generation),391 Series C32F391 32 16:9 Curved FreeSync LCD Monitor,3TB AirPort Time Capsule (5th Generation),450 Mbps 2.4/5GHz Wireless N Dual Band PCI Express Adapter,480GB Extreme Pro Solid State Drive,4GB NW-WS413 Sports Walkman Digital Music Player (Black),...,Yamaha - WX-010 Wireless Speaker for Streaming Music - Black,Yamaha R-N301 Network Audio/Video Player - Black (rn301bl),Yamaha RX-V379BL 5.1-Channel AV Receiver (Black),Yamaha RX-V383BL 5.1-Channel 4K Ultra HD AV Receiver with Bluetooth,Yamaha RX-V479BL 5.1-Channel AV Receiver (Black),Yamaha RX-V681BL 7.2-Channel MusicCast AV Receiver with Bluetooth,Yamaha WX-030BL MusicCast Wireless Speaker with Wi-Fi and Bluetooth (Black),ZS-RS60BT CD Boombox,ZTE - Spro 2 Wireless Smart DLP Projector - Silver,ZTE Warp Elite No Contract Phone - Retail Packaging - Boost,Zivix - puc+ Bluetooth MIDI Interface - Black,Zoom - H1 Handy Recorder - Black,Zubie - In-Car Wi-Fi and Vehicle Monitoring Device - Black,e1659Fwu 16 Widescreen USB 3.1 Gen 1 Powered LED Backlit LCD Monitor,fr Case for Galaxy S7 (Black),h.ear go Wireless Speaker (Bordeaux Pink),h.ear go Wireless Speaker (Cinnabar Red),h.ear go Wireless Speaker (Lime Yellow),h.ear go Wireless Speaker (Viridian Blue),i.am+ BUTTONS,iHome Rechargeable Splash Proof Stereo Bluetooth Speaker - Black (IBT33BC),iHome iBN43BC Bluetooth Stereo Dual Alarm FM Clock Radio and Speakerphone with USB Charging,iLive - ISBW216 Wireless Gnome Outdoor Speaker - Multi,iPL24 FM Clock Radio with Lightning Dock and USB Port (Gunmetal),iPhone 7 Smart Battery Case (Black),iSimple - BluStream Bluetooth Factory Radio Module - Black,"iSimple - StrongHold Headrest Mount for Most 7 - 10.2"" Tablets - Black""",iSimple - TranzIt Bluetooth Factory Radio Module - Black,iSimple ISBC01 BluClik Bluetooth Remote Control with Steering Wheel and Dash Mounts,inactive,instax mini 8 Instant Film Camera and Groovy Case Kit (Raspberry),instax mini Rainbow Instant Film (10 Exposures),j5create - VGA to HDMI Video adapter - White,"kate spade new york - Glitter Sleeve for 13 Apple® MacBook® - Rose Gold""",kenwood pkg-mr362bt marine cd receiver with bluetooth and 6.5 2 way speakers package,mophie - Juice Pack External Battery Case with Wireless Charging for Apple® iPhone® 7 Plus and 8 Plus - Rose Gold,mophie powerstation Plus Mini External Battery with Built in Cables for Smartphones and Tablets (4,myCharge - RAZOR PLUS USB Portable Power Bank - Si

In [18]:
x_new_["BOOM 2 Wireless Bluetooth Speaker - Indigo"].value_counts()

0.0    19
Name: BOOM 2 Wireless Bluetooth Speaker - Indigo, dtype: int64

 ## Price elasticity = Change in Quanity sold / Change in the Price

In [62]:
#Initialization
final_df = pd.DataFrame()


In [63]:
def multizen(cox):
    global final_df

    final_values = {"category": [], "name": [], "price_elasticity": [], "price_mean": [], "impressions_mean": [], "intercept": [], "slope": []}

    temp_df = pd.DataFrame()
    temp_df['x'] = x_new_[cox] #price
    temp_df['y'] = y_new_[cox].pct_change()
    temp_df.fillna(method='bfill', inplace=True)
    
    x_value = temp_df[['x']]
    y_value = temp_df['y']

    mean_price = np.mean(x_value)
    mean_imp = np.mean(y_value)


    if mean_price[0] == 0:
        return
    
    if mean_imp == 0:
        return

    #Creating Linear Model
    model = LinearRegression()
    model.fit(x_value, y_value)

    #Eval
    slope = model.coef_ 
    intercept = model.intercept_

    #####
    #y_pred = model.predict(x_value)
    #r_squared = r2_score(y_value, y_pred)

    #Price elasticity
    price_elasticity = slope*(mean_price/mean_imp)
    #price_elasticity = -model.coef_[0] * (mean_price/mean_imp)


    #To find Category
    category = dfx_mod.loc[dfx_mod['Name'] == cox, 'Category_name'].iloc[0]

    #Append results 
    final_values["category"].append(category)
    final_values["name"].append(cox)
    final_values["price_elasticity"].append(price_elasticity[0])
    final_values["price_mean"].append(mean_price[0])
    final_values["impressions_mean"].append(mean_imp)
    final_values["intercept"].append(intercept)
    final_values["slope"].append(slope[0])

    final_df = pd.concat([final_df,pd.DataFrame.from_dict(final_values)],axis=0,ignore_index=True)
    



In [64]:
# Running Model for every Product.

for col in x_new_.columns[1:]:
    multizen(col)

    

#### Final DF

In [65]:
final_df

,category,name,price_elasticity,price_mean,impressions_mean,intercept,slope
0,"laptop, computer",12.3 32GB Multi-...,1.133544,3.587719,0.465692,-0.062191,0.147136
1,"tablet, computer...",1500-Watt Digita...,-0.122007,1.482456,1.385581,1.554631,-0.114034
2,"tablet, computer...",1506TSA Lock,0.323839,1.394737,0.560388,0.378912,0.130115
3,"car, speaker, su...",2) New Kicker 41...,-0.811909,36.819549,1.014485,1.838155,-0.022370
4,"monitor, computer",22 Widescreen Fu...,1.177590,1.108647,0.568914,-0.101033,0.604293
...,...,...,...,...,...,...,...
419,"car, gps, dash",iSimple ISBC01 B...,0.243295,0.236842,0.092949,0.070335,0.095481
420,"flash, camera",inactive,-0.187500,0.852632,0.074251,0.088173,-0.016328
421,"camera, shoot, p...",instax mini 8 In...,-0.403817,2.947368,-0.162028,-0.227458,0.022199
422,"power, battery, ...",mophie powerstat...,-0.121407,1.438596,4.779860,5.360170,-0.403386


In [66]:
#Exporing the Final Data as CSV
final_df.to_csv("xFinalData4EachProduct.csv",index=False)

# Feature Selection Methods

#### Forward Selection Method

In [24]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=3,direction="forward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [0 2 3]


#### Backward Selection Method

In [25]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data
y = iris.target

lr = LogisticRegression()
sfs = SequentialFeatureSelector(lr, n_features_to_select=2,direction="backward")

sfs.fit(X, y)

sfs.get_support()
sfs.transform(X).shape
print('Selected features:', sfs.get_support(indices=True))

Selected features: [2 3]


#### Based on P Value

In [26]:
from sklearn.feature_selection import f_regression

# load the Iris dataset
iris = load_iris()

# extract the features and target variable
X = iris.data
y = iris.target

# calculate the F-test statistic and p-value for each feature
f_test, p_values = f_regression(X, y)

# print the p-value of each feature
for i, p_value in enumerate(p_values):
    print(f"Feature {i+1}: p-value = {p_value:.4f}")


# if p < 0.05

Feature 1: p-value = 0.0000
Feature 2: p-value = 0.0000
Feature 3: p-value = 0.0000
Feature 4: p-value = 0.0000


#### Variance Inflastion Factor (VIF)

In [27]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# load the Iris dataset
iris = load_iris()

# extract the features and target variable
X = iris.data
y = iris.target
X =pd.DataFrame(X)

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

# Print results
print(vif)


   VIF Factor  features
0  262.969348         0
1   96.353292         1
2  172.960962         2
3   55.502060         3


In [28]:
# Feature with High VIF factor is likely to be highly correlated with the other features.

#### ANOVA F-value (p)

ANOVA (Analysis of Variance) F-value is used to evaluate the relevance of each feature to the target variable in a classification or regression task.

In [29]:
from sklearn.feature_selection import SelectKBest, f_classif

# load the Iris dataset
iris = load_iris()

# extract the features and target variable
X = iris.data
y = iris.target
X =pd.DataFrame(X)

# ANOVA  F-value 
selector  = SelectKBest(f_classif, k=3)

selector.fit(X, y)

# Print results
print('Selected features:', selector.get_support(indices=True))

Selected features: [0 2 3]


The negative sign in the formula for price elasticity of demand (PED) is used to indicate the inverse relationship between the price of a good or service and the quantity demanded by consumers. In other words, as the price of a good or service increases, the quantity demanded by consumers decreases, and vice versa.

The PED formula is:

PED = (% change in quantity demanded / % change in price)

By convention, the price elasticity of demand is expressed as a negative number, as the percentage change in quantity demanded and price usually have opposite signs. For example, if the price of a good increases by 10%, the quantity demanded may decrease by 5%, resulting in a PED of -0.5. The negative sign indicates that the demand for the good is responsive to changes in price and that the quantity demanded decreases as the price increases.

However, in some cases, the demand for a good may be inelastic, which means that the PED is less than 1. In this case, the percentage change in quantity demanded may be smaller than the percentage change in price, resulting in a positive PED value. But, in general, the negative sign is used to indicate the inverse relationship between price and quantity demanded.